In [1]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../../quant_container/src')
sys.path.append('../src')
%load_ext autoreload
%autoreload 2
from mosaicsmartdata.core.fx import FXPricingContextGenerator
from mosaicsmartdata.common import qc_csv_helper
from mosaicsmartdata.core.quote import Quote
import csv, copy
import pickle

In [2]:
# try:
#     with open('data.pickle', 'rb') as f:
#         # Pickle the 'data' dictionary using the highest protocol available.
#         quote_dict = pickle.load(f)
# except:
if True:
    file_path ='../resources/fx/'
    filenames = ['OIS-N156898469.csv','EUR_JPY_GBP_till_SN.csv','EUR_JPY_GBP_from_SW_till_1Y.csv']
    quote_dict_0 = qc_csv_helper.get_ric_quotes(file_path+filenames[0])
    print(len(quote_dict_0), quote_dict_0.keys())
    quote_dict_1 = qc_csv_helper.get_ric_quotes(file_path+filenames[1])
    print(len(quote_dict_1), quote_dict_1.keys())
    quote_dict_2 = qc_csv_helper.get_ric_quotes(file_path+filenames[2])
    print(len(quote_dict_2), quote_dict_2.keys())
    quote_dict = dict(quote_dict_0,**quote_dict_1)
    quote_dict = dict(quote_dict,**quote_dict_2)
    with open('data.pickle', 'wb') as f:
        pickle.dump(quote_dict, f)

6410 <class 'mosaicsmartdata.core.quote.Quote'> :  duration = None, sym = USD1YOIS=, ask = 1.296, bid = 1.276, timestamp = 2017-06-30 20:50:34.174000, units = percent, instrument = <class 'mosaicsmartdata.core.instrument.OIS'> :  ccy = USD, sym = USD1YOIS=, tenor = None, maturity_date = 2018-07-05, spot_settle_date = 2017-07-05, par_value = 100, holidayCities = None, coupon = None, float_coupon_frequency = None, duration = None, issue_date = None, venue = None, price_type = None, country_of_risk = (236,), day_count = None, coupon_frequency = None
5 dict_keys(['USD1MOIS=', 'USD6MOIS=', 'USD1YOIS=', 'USD3MOIS=', 'USDSWOIS='])
100000 <class 'mosaicsmartdata.core.quote.Quote'> :  duration = None, sym = EUR=, ask = 1.119, bid = 1.1189, timestamp = 2017-06-27 03:11:19.399000, units = outright, instrument = <class 'mosaicsmartdata.core.instrument.FXForward'> :  isDeliverable = True, static = Instrument static singleton facade, sym = EUR=, ccy = ('EUR', 'USD'), notionals = None, date_calc = <m

1500000 <class 'mosaicsmartdata.core.quote.Quote'> :  duration = None, sym = GBP=, ask = 1.2813, bid = 1.2812, timestamp = 2017-06-28 06:01:26.157000, units = outright, instrument = <class 'mosaicsmartdata.core.instrument.FXForward'> :  isDeliverable = True, static = Instrument static singleton facade, sym = GBP=, ccy = ('GBP', 'USD'), notionals = None, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x00000238C7032BA8>, settle_date = 2017-06-30, pip_size = 0.0001, venue = None, holidayCities = None, tenor = SPOT, maturity_date = 2017-06-30
1600000 <class 'mosaicsmartdata.core.quote.Quote'> :  duration = None, sym = GBP=, ask = 1.2931, bid = 1.2928, timestamp = 2017-06-28 22:13:11.734000, units = outright, instrument = <class 'mosaicsmartdata.core.instrument.FXForward'> :  isDeliverable = True, static = Instrument static singleton facade, sym = GBP=, ccy = ('GBP', 'USD'), notionals = None, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator objec

1100000 <class 'mosaicsmartdata.core.quote.Quote'> :  duration = None, sym = JPY1Y=, ask = -199.23, bid = -203.85, timestamp = 2017-06-26 16:21:37.741000, units = pips, instrument = <class 'mosaicsmartdata.core.instrument.FXSwap'> :  sym = JPY1Y=, ccy = ('USD', 'JPY'), legs = [<mosaicsmartdata.core.instrument.FXForward object at 0x00000238ED91B3C8>, <mosaicsmartdata.core.instrument.FXForward object at 0x00000238ED91B630>], venue = None, holidayCities = None, tenor = 1Y, settle_date = 2017-06-28, maturity_date = 2018-06-28
1200000 <class 'mosaicsmartdata.core.quote.Quote'> :  duration = None, sym = JPY1Y=, ask = -202.75, bid = -206.26, timestamp = 2017-06-29 14:42:54.747000, units = pips, instrument = <class 'mosaicsmartdata.core.instrument.FXSwap'> :  sym = JPY1Y=, ccy = ('USD', 'JPY'), legs = [<mosaicsmartdata.core.instrument.FXForward object at 0x00000238F0375EB8>, <mosaicsmartdata.core.instrument.FXForward object at 0x00000238F0375048>], venue = None, holidayCities = None, tenor = 1

In [3]:
quote_dict['EURSN='][0].instrument.settle_date

datetime.date(2017, 6, 28)

In [4]:
# create a trade
from mosaicsmartdata.core.instrument_utils import sym_to_instrument
from mosaicsmartdata.core.trade import Trade, FXForwardTrade

#spot_quotes = [quotes for sym, quotes in quote_dict.items() if len(sym)==4]
my_quote = quote_dict['EUR='][100]
my_instr = sym_to_instrument()(my_quote.sym, my_quote.timestamp.date())
my_instr.notionals = (100, -100*my_quote.bid)
print(my_instr)
print('*****')
my_trade = FXForwardTrade(my_instr, 
                          trade_id = '12345',
                        timestamp = my_quote.timestamp)
print(my_trade)
print('*******')
print(my_quote)

<class 'mosaicsmartdata.core.instrument.FXForward'> :  isDeliverable = True, static = Instrument static singleton facade, sym = EUR=, ccy = ('EUR', 'USD'), notionals = (100, -111.97999999999999), date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x00000238A08E59B0>, settle_date = 2017-06-28, pip_size = 0.0001, venue = None, holidayCities = None, tenor = SPOT, maturity_date = 2017-06-28
*****
<class 'mosaicsmartdata.core.trade.FXForwardTrade'> :  orig_package_size = None, trader = None, notional = None, factor_risk = None, paper_trade = False, delta = 1.0, timestamp = 2017-06-26 00:02:26.995000, trade_rc = None, sales = None, traded_px = 1.1198, package = None, adj_traded_px = None, trade_id = 12345, instrument = <class 'mosaicsmartdata.core.instrument.FXForward'> :  isDeliverable = True, static = Instrument static singleton facade, sym = EUR=, ccy = ('EUR', 'USD'), notionals = (100, -111.97999999999999), date_calc = <mosaicsmartdata.core.date_calculator.DateCal

In [5]:
from aiostreams import run
import aiostreams.operators as op
from mosaicsmartdata.core.markout import MarkoutCalculator
import logging

In [6]:
# run markouts based on simple instrument ID, with no pricing context
# logging.getLogger().setLevel('INFO')
# logging.info('starting command line test...')

# run the markout calc
trades = [my_trade]
quotes = quote_dict['EUR=']
stream = op.merge_sorted([quotes[:1000], trades], lambda x: x.timestamp) \
            | op.flat_map_by_group(lambda x: x.sym, MarkoutCalculator([-10, 0, 10, 100]))\
            > print
        
run(stream)

 dt:-10 final_price:1.1199 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:16.995000 package:None PV:9.999999999998899e-05 price:9.999999999998899e-05 timestamp:2017-06-26 00:02:26.995000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None trade_maturity_date:28/06/2017 traded_px:1.1198 ccy:('EUR', 'USD')
 dt:0 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:26.995000 package:None PV:5.0000000000105516e-05 price:5.0000000000105516e-05 timestamp:2017-06-26 00:02:27.165000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None trade_maturity_date:28/06/2017 traded_px:1.1198 ccy:('EUR', 'USD')
 dt:10 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_time

In [7]:
# run markouts using PricingContext
# run the markout calc

quotes_eur = quote_dict['EUR=']
quotes_jpy = quote_dict['JPY=']
print(len(quotes_jpy),len(quotes_eur))
quotes_all = op.merge_sorted([quotes_eur[:1000],quotes_jpy[:1000]], lambda x: x.timestamp) > []
             #|\op.map(lambda x: (x.sym, x.timestamp)) > 
run(quotes_all)

my_quote = quotes_all.sink[100]
my_instr_2 = sym_to_instrument()('EURJPY=', my_quote.timestamp.date())
my_instr_2.notionals = (100, -100*124.594511) # just picked a random quote
print(my_instr_2) 
print('*****')
my_trade_2 = FXForwardTrade(my_instr_2, 
                          trade_id = '12346',
                        timestamp = my_quote.timestamp)
print(my_trade_2.timestamp)


553108 591602
<class 'mosaicsmartdata.core.instrument.FXForward'> :  isDeliverable = True, static = Instrument static singleton facade, sym = EURJPY=, ccy = ('EUR', 'JPY'), notionals = (100, -12459.4511), date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x0000023905555E80>, settle_date = 2017-06-28, pip_size = 0.0001, venue = None, holidayCities = None, tenor = SPOT, maturity_date = 2017-06-28
*****
2017-06-26 00:00:53.188000


In [8]:
#markouts using PricingContext
trades = [my_trade]
quotes_2 = quotes_all.sink
#print([quote.sym for quote in quotes_2][:100])

prices =  quotes_2 | op.map(FXPricingContextGenerator()) 
#stream = prices | op.map(lambda x: x.timestamp) > print
stream = op.merge_sorted([trades,prices], lambda x:x.timestamp)\
        | op.flat_map(MarkoutCalculator([-10,0, 10, 100])) | op.map(lambda x:str(x))\
        > []
run(stream)       

stream.sink

'list' object has no attribute 'timestamp'
Traceback (most recent call last):
  File "../../quant_container/src\aiostreams\runner.py", line 161, in check_tasks
    t.result()
  File "C:\Users\egork\Anaconda3\envs\py35\lib\asyncio\futures.py", line 293, in result
    raise self._exception
  File "C:\Users\egork\Anaconda3\envs\py35\lib\asyncio\tasks.py", line 239, in _step
    result = coro.send(None)
  File "../../quant_container/src\aiostreams\runner.py", line 58, in coro
    async for msg in self.source:
  File "../../quant_container/src\aiostreams\operators\map.py", line 12, in __anext__
    val = await self.aiter.__anext__()
  File "../../quant_container/src\aiostreams\operators\flatten.py", line 18, in __anext__
    self.latest = await self.aiter.__anext__()
  File "../../quant_container/src\aiostreams\operators\map.py", line 12, in __anext__
    val = await self.aiter.__anext__()
  File "../../quant_container/src\aiostreams\operators\merge_sorted.py", line 56, in __anext__
    tes

[]

In [9]:
# extract just the first quote from every stream
tiny_quote_dict = {key:[quotes[0]] for key, quotes in quote_dict.items()}
print([(key,quotes[0].bid, quotes[0].instrument.ccy) for key, quotes in tiny_quote_dict.items()])
with open('../resources/fx/data.pickle', 'wb') as f:
        pickle.dump(tiny_quote_dict, f)


[('EUR6M=', 110.79, ('EUR', 'USD')), ('EUR3M=', 54.91, ('EUR', 'USD')), ('JPY=', 111.22, ('USD', 'JPY')), ('EURON=', 0.525, ('EUR', 'USD')), ('GBPTN=', 0.385, ('GBP', 'USD')), ('GBPON=', 0.352, ('GBP', 'USD')), ('GBP=', 1.2744, ('GBP', 'USD')), ('JPYON=', -0.47, ('USD', 'JPY')), ('JPY1Y=', -202.86, ('USD', 'JPY')), ('EURTN=', 0.51, ('EUR', 'USD')), ('GBP1Y=', 141.19, ('GBP', 'USD')), ('EUR=', 1.1198, ('EUR', 'USD')), ('USD3MOIS=', 1.148, 'USD'), ('JPYTN=', -0.481, ('USD', 'JPY')), ('JPYSW=', -5.7, ('USD', 'JPY')), ('GBPSW=', 4.53, ('GBP', 'USD')), ('EUR1Y=', 230.8, ('EUR', 'USD')), ('USD1MOIS=', 1.141, 'USD'), ('GBP6M=', 70.3, ('GBP', 'USD')), ('USD6MOIS=', 1.175, 'USD'), ('JPYSN=', -0.466, ('USD', 'JPY')), ('JPY1M=', -16.39, ('USD', 'JPY')), ('USDSWOIS=', 1.11, 'USD'), ('GBPSN=', 0.387, ('GBP', 'USD')), ('EURSN=', 0.54, ('EUR', 'USD')), ('EUR1M=', 19.11, ('EUR', 'USD')), ('JPY6M=', -93.41, ('USD', 'JPY')), ('USD1YOIS=', 1.24, 'USD'), ('EURSW=', 5.84, ('EUR', 'USD')), ('GBP3M=', 36.12,

In [10]:
import datetime
a = datetime.date(2107,1,1)
b = datetime.date(2107,1,2)
from mosaicsmartdata.common.quantlib.bond.fixed_bond import pydate_to_qldate
pydate_to_qldate(b)< pydate_to_qldate(a)


False

In [15]:
with open('../resources/fx/data.pickle', 'rb') as f:
        tiny_quote_dict = pickle.load(f)

        
tiny_quotes_list = [quotes for key, quotes in tiny_quote_dict.items()]
print([q[0].instrument.sym for q in tiny_quotes_list])
stream = op.merge_sorted(tiny_quotes_list, lambda x: x.timestamp) |\
            op.flat_map(FXPricingContextGenerator()) > []
run(stream)
c = stream.sink[-1]
c.curve

['EUR6M=', 'EUR3M=', 'JPY=', 'EURON=', 'GBPTN=', 'GBPON=', 'GBP=', 'JPYON=', 'JPY1Y=', 'EURTN=', 'GBP1Y=', 'EUR=', 'JPY3M=', 'JPY1M=', 'JPYTN=', 'GBPSW=', 'EUR1Y=', 'USD1MOIS=', 'GBP6M=', 'USD6MOIS=', 'JPYSN=', 'USD3MOIS=', 'USDSWOIS=', 'GBPSN=', 'EURSN=', 'EUR1M=', 'JPY6M=', 'USD1YOIS=', 'EURSW=', 'GBP3M=', 'GBP1M=', 'JPYSW=']


{'EUR': <mosaicsmartdata.common.quantlib.curve.usdois.USDOIS at 0x23916a5a358>,
 'GBP': <mosaicsmartdata.common.quantlib.curve.usdois.USDOIS at 0x23929d2beb8>,
 'JPY': <mosaicsmartdata.common.quantlib.curve.usdois.USDOIS at 0x23916a5af28>,
 'USD': <mosaicsmartdata.common.quantlib.curve.usdois.USDOIS at 0x23892ff7e10>}

In [12]:

a = 2
b = 3
a,b = b,a
a

3